In [12]:
"""
Mount the remote disk to work with data.


sshfs -o allow_other,defer_permissions \
USER@matlaber1.media.mit.edu:/REMOTE_DATA_PATH \
/LOCAL_DATA_PATH/

later...
umount /LOCAL_DATA_PATH/

"""
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# How is this data different than that in other files, for different MSA's or months?
MSA = "14460"
TIME_PERIOD = "201805"

data_path = "../data/mount/{time_period}/".format(time_period=TIME_PERIOD)
filename = "{msa}_{time_period}.csv".format(msa=MSA, time_period=TIME_PERIOD)
filepath = data_path + filename
print(filepath)

../data/mount/201805/14460_201805.csv


In [13]:
# Define variables that will be used at both entry and re-entry parts of script

DEVICE_ID = 'device ID'
DEVICE_ID_OCCURANCES = 'device ID occurances'
DAY = 'day'
DEVICE_ID_DAYS_COUNT = 'days count'

DWELLTIME = 'DwellTime'
TIMESTAMP = 'timestamp'

In [3]:
"""Utility for testing w. lower bandwidth: read in just a random sample of data.

NOTE: can only do this for scratch/testing code, etc.  The data will lose a lot
because this is not a random sample of USERs - it's a random sample of data points
"""
import random

DEFAULT_RANDOM_SAMPLE_PORTION = 0.01 # to keep random % of the lines

def read_csv_random_sample(filepath, portion=DEFAULT_RANDOM_SAMPLE_PORTION):
    return pd.read_csv(
         filepath,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > portion
    )

In [4]:
# FOR TESTING / DEBUGGING
# reading in small random sample of file
df = read_csv_random_sample(filepath)
print(df.shape)

(86695, 7)


In [28]:
# Read in the data of interest. (All of it)
df = pd.read_csv(filepath)
print(df.shape)

(8680859, 7)


In [29]:
# update the columns to use 'device ID instead of LBS provider name'
df.rename(columns={df.columns[2]:DEVICE_ID}, inplace=True)

In [30]:
"""Filter out garbage / outlier datapoints."""

# Filter out rows where dwelltime > MAX_DWELLTIME
MAX_DWELLTIME = 24*60 # 24 hours
row_count_before = df.shape[0]
df = df[df[DWELLTIME] <= MAX_DWELLTIME]
rows_dropped = (row_count_before - df.shape[0])
print('dropped %s of %s rows where dwelltime > maximum of %s' % (rows_dropped, row_count_before, MAX_DWELLTIME))

dropped 166686 of 8680859 rows where dwelltime > maximum of 1440


In [31]:
"""Filter the data to users who have enough data points.
Filter data to users with:
- data points for (>15 days) at least half the days of the month
- enough data for each day: avg of 2 datapoints for each day they are represented

(0) Preliminary step:
Filter to users with at least USER_TOTAL_POINTS_THRESHOLD (32) datapoints
- find # of occurances for each user ID
- filter out users with < THRESHOLD # occurances

"""

USER_TOTAL_DAYS_THRESHOLD = 16
USER_TOTAL_OCCURANCES_THRESHOLD = 32 # require at least avg of 2 points per day (where # days at least 16)


unique_device_IDs = df[DEVICE_ID].nunique()
print('unique device ids:', unique_device_IDs)
df.sort_values(by=[DEVICE_ID], inplace=True)
# df.head() # check: yeah that looks right

unique device ids: 250857


In [32]:
# Add value counts column to DF.
# i.e. add counts for number of occurances of given device ID
df[DEVICE_ID_OCCURANCES] = df[DEVICE_ID].map(df[DEVICE_ID].value_counts())

# drop rows where value count for that row's device ID < USER_TOTAL_POINTS_THRESHOLD
row_count_before = df.shape[0]
user_count_before = df[DEVICE_ID].nunique()
print('row count before: ', row_count_before)
print('user count before: ', user_count_before)
df = df[df[DEVICE_ID_OCCURANCES] >= USER_TOTAL_OCCURANCES_THRESHOLD]
df.sort_values(by=[DEVICE_ID_OCCURANCES, DEVICE_ID], ascending=True, inplace=True)


# Add count of how many unique days on which this user is represented in the dataset
# map device ID to # unique days
DEVICE_ID_to_days_df = df.groupby(DEVICE_ID)[DAY].nunique()
df[DEVICE_ID_DAYS_COUNT] = df[DEVICE_ID].map(DEVICE_ID_to_days_df)
# before dropping, check:
# check: Does this look right?  sort the data by ascending number of days represented, and take a look
df.sort_values(by=[DEVICE_ID_DAYS_COUNT, DEVICE_ID], inplace=True)
# drop rows for users with less than USER_TOTAL_DAYS_THRESHOLD
df = df[df[DEVICE_ID_DAYS_COUNT] >= USER_TOTAL_DAYS_THRESHOLD]
# how much did we drop?
rows_dropped = row_count_before - df.shape[0]
users_dropped = user_count_before - df[DEVICE_ID].nunique()
print('rows dropped: ', rows_dropped, '; remaining rows:', df.shape[0])
print('users dropped: ', users_dropped, '; remaining users:', len(df[DEVICE_ID].unique()))

row count before:  8514173
user count before:  250857
rows dropped:  2603129 ; remaining rows: 5911044
users dropped:  200118 ; remaining users: 50739


In [ ]:
# Save this filtered dataset
filtered_filepath = data_path + "filtered/" + "filtered_" + filename
df.to_csv(filtered_filepath)
# This is a stopping point - you can come back and read the data in from here.

# Re-Entry Point for already filtered data


In [ ]:
filtered_filepath = data_path + "filtered/" + "filtered_" + filename

In [ ]:
# FOR TESTING / DEBUGGING
# Read small random sample of previously filter data
df = read_csv_random_sample(filtered_filepath)

In [ ]:
# Read all previously filtered data
df = pd.read_csv(filtered_filepath)
print(df.shape)

In [ ]:
# Limit data to random sample of users (users not datapoints)

sample_portion = 0.3

user_ids = df[DEVICE_ID].unique()
sample_count = int(len(user_ids)*sample_portion)
sampled_user_ids = random.sample(list(user_ids), sample_count)
print("keeping only {sample}% sample of users datapoints".format(sample=sample_portion*100))
df = df[df[DEVICE_ID].isin(sampled_user_ids)]
print("%s remaining users" % sample_count)
# df.head()

# Infer Home Location (as census area)

To infer home census area (tract/block group)s:

make a DF where its the same data, but limited to nighttime points
- Define a time range for night time (in hours)
- Add a column for 'nighttime dwelltime' as the dwelltime that occurs within those nighttime hours
    - i.e. if datapoint has a dwelltime of 3 hours starting at 7pm, while starting nighttime hour is 8pm then nighttime dwelltime for this datapoint is 2 hours

make nighttime DF as the subset of data where the nighttime dwelltime > 0

filter out users for whome there are less than required threshold of nighttime datapoints

assign census area (tract/block group) to each point in nighttime df
- lat,lon point falls within area --> row assigned to area
- this is added as a column

infer home geoid as area where user has largest nighttime dwelltime

In [ ]:
"""
Notes on Working with timestamps.
The timestamps are in ISO format.  e.g. '2018-05-01T07:18:02-04:00'
The offset refers to how many hours the timezone is from Coordinated Universal Time (UTC).
"""
from datetime import datetime, timedelta
from dateutil import tz, parser


# columns I add:
# local time for when dwell starts and ends
DWELLTIME_START = 'dwelltime start'
NIGHTTIME_DWELLTIME = 'nighttime dwelltime'


NIGHTTIME_START_HOUR = 20 # 8pm
NIGHTTIME_END_HOUR = 8


def timedelta_minutes(tdelta):
    """Helper function. Returns minutes for timedelta object."""
    return round((tdelta.seconds / 60), 2)

# tests for the timedelta_minutes function
assert(timedelta_minutes(timedelta(minutes=230.95)) == 230.95)
assert(timedelta_minutes(timedelta(minutes=3.37)) == 3.37)


# add local start for dwelltime
def dwelltime_start(tstamp):
    # tstamp may be a datettime.time or str -- handle either
    return parser.parse(str(tstamp))

df[DWELLTIME_START] = df[TIMESTAMP].apply(dwelltime_start)


# calculate nighttime dwelltime
def nighttime_dwelltime(row):
    """Returns the portion of the dwelltime that occurred in the nighttime hours."""
    dwelltime = timedelta(minutes=float(row[DWELLTIME]))
    dwell_start = row[DWELLTIME_START]
    dwell_start_hour = dwell_start.time().hour
    dwell_end = dwell_start + dwelltime
    dwell_end_hour = dwell_end.time().hour
    # Return zero for dwelltimes that do not overlap with the nighttime hours.
    if (dwell_start_hour > NIGHTTIME_END_HOUR) \
        and (dwell_start_hour < NIGHTTIME_START_HOUR) \
        and (dwell_end_hour < NIGHTTIME_START_HOUR) \
        and (dwell_end_hour > NIGHTTIME_END_HOUR):
        return 0
    
    if (dwell_start.time().hour <= NIGHTTIME_END_HOUR):
        # dwelltime starts in the early hours of the morning, before nighttime ends.
        # restrict the dwelltime to the nighttime hours
        nighttime_end_dt = datetime(dwell_start.year,  dwell_start.month, dwell_start.day, hour=NIGHTTIME_END_HOUR, tzinfo=dwell_start.tzinfo)
        nighttime_dwelltime = min((nighttime_end_dt - dwell_start), dwelltime)
    
    else: # otherwise... the dwelltime starts before midnight
        nighttime_dwelltime = dwelltime
        # if it start before the nighttime starts, subtract the portion before nighttime starts
        nighttime_start_dt = datetime(dwell_start.year,  dwell_start.month, dwell_start.day, hour=NIGHTTIME_START_HOUR, tzinfo=dwell_start.tzinfo)
        if dwell_start < nighttime_start_dt:
            delta = nighttime_start_dt - dwell_start
            nighttime_dwelltime -= delta
        
        # if the dwell continues until the next day, and
        # if the dwell ends after the nighttime ends,
        # then subtract the portion that happens after nighttime ends
        if ((dwell_end_hour < NIGHTTIME_START_HOUR) and (dwell_end_hour > NIGHTTIME_END_HOUR)):
            # the nighttime end for this case is the next day
            nighttime_end_dt = datetime(dwell_end.year,  dwell_end.month, dwell_end.day, hour=NIGHTTIME_END_HOUR, tzinfo=dwell_end.tzinfo)
            delta = dwell_end - nighttime_end_dt
            nighttime_dwelltime -= delta
    return timedelta_minutes(nighttime_dwelltime)
        
    
# add nighttime dwelltime
df[NIGHTTIME_DWELLTIME] = df[[DEVICE_ID, TIMESTAMP, DWELLTIME, DWELLTIME_START]].apply(nighttime_dwelltime, axis=1)


In [ ]:
# Make additional DF limited to nighttime datapoints 
print('df all rows count:', df.shape[0])
print('df unique users count:', df[DEVICE_ID].nunique())
nighttime_df = df[df[NIGHTTIME_DWELLTIME] > 0].copy()
print('nighttime df rows count:', nighttime_df.shape[0])
print('nighttime df unique users count:', nighttime_df[DEVICE_ID].nunique())
# nighttime_df.head()

In [ ]:
# Remove the users for whom there are fewer than threshold of nighttime data points
# (do this filtering before the computationally expensive process of putting points in census areas)

# What is the right threshold?
USER_NIGHTTIME_POINTS_THRESHOLD = 4

user_ids = nighttime_df[DEVICE_ID].unique()

user_nights = nighttime_df.groupby([DEVICE_ID, DAY])[DEVICE_ID].count().groupby(level=DEVICE_ID).count()
keep_user_ids = user_nights[user_nights >= USER_NIGHTTIME_POINTS_THRESHOLD].index.tolist()

print('filtered out users with fewer than %s unique nights of data' % USER_NIGHTTIME_POINTS_THRESHOLD)
print('user count before:', len(user_ids), '\nuser count after filtering:', len(keep_user_ids))
nighttime_df = nighttime_df[nighttime_df[DEVICE_ID].isin(keep_user_ids)]
print('remaining nighttime datapoints: ', nighttime_df.shape[0])

In [ ]:
# assign census tract/block to each point in nighttime df
# - lat,lon point falls within area --> row assigned to area
# - this is added as a column
#
# BUT FIRST: filter out points that do not fall within target counties
# map geolocation to countysub & county within shapefile
# The shapefile is for a limited set of counties that contain target census tracts
# reason this is good filtering:
# - fewer county shapes than tracts for given area, so less computationally intensive
# - then filter out data that did not fall within target counties (i.e. in other parts of MSA)

from datetime import datetime
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

COUNTY_FP = "COUNTYFP"
COUNTY_SUBDIVISION_FP = "COUSUBFP"
print('NOTE: using limited area of boston, brookline, cambridge, somerville, so most datapoints will not fall in this area')
county_subdivs_shapefile_filepath = "./shapefiles/ma/boston-brookline-cambridge-somerville_countysubdivisions.shp"
county_subdivs_shapefile = geopandas.read_file(county_subdivs_shapefile_filepath)
county_subdivs_shapefile = county_subdivs_shapefile.to_crs(epsg=4326)
county_subdivs_shapefile.plot()

def get_county_subdiv(row):
    r = random.randint(1, 10000)
    if r == 1:
        print(datetime.now(), "get county subdiv")
    lat = float(row['lat'])
    lon = float(row['lon'])
    point = Point(lon, lat) # the points actually reverse - yes this looks weird but consider lon the x-axis and lat the y-axis on an x-y plane...
    for i, row in county_subdivs_shapefile.iterrows():
        try:
            # this could be a Polygon or a Multipolygon when the tract contains islands (<3 New England)
            polygon = row['geometry']
            if point.within(polygon):
                return row[COUNTY_SUBDIVISION_FP]
        except Exception as e:
            print('Exception when looking for point in county subdivision ' + row[COUNTY_SUBDIVISION_FP], ': ', e)
            raise e
    return None

nighttime_df[COUNTY_SUBDIVISION_FP] = nighttime_df.apply(get_county_subdiv, axis=1)
nighttime_df.head()

In [ ]:
filtered_nighttime_df = nighttime_df

In [ ]:
# Drop the rows for datapoints where census area could not be determined
rows_before = filtered_nighttime_df.shape[0]
users_count_before = filtered_nighttime_df[DEVICE_ID].nunique()
filtered_nighttime_df.dropna(inplace=True)
rows_after = filtered_nighttime_df.shape[0]
users_count_after = filtered_nighttime_df[DEVICE_ID].nunique()
print("dropped %s rows where county subdivision not determined. rows remaining: %s" % (rows_before - rows_after, rows_after))
print("users before: %s \nusers after: %s" % (users_count_before, users_count_after))

# filtered_nighttime_df.head()

# Save Filtered Data Checkpoint

In [ ]:
# Save this filtered dataset

filtered_nighttime_filepath = data_path + "filtered/" + "filtered_" + "{msa}_{time_period}_nighttime_bos_brook_cam_som.csv".format(msa=MSA, time_period=TIME_PERIOD)
print("saving data to file", filtered_nighttime_filepath)
filtered_nighttime_df.to_csv(filtered_nighttime_filepath)
# This is a stopping point - you can come back and read the data in from here.

In [ ]:
"""
Attach census block group and census tract information to each datapoint.
A census tract contains many block groups.
The GEOID for a block group is 12 digits, where the GEOID for a census tract is 11 digits.
The first 11 digits in a block group are the census tract.
Thus a single function can determine both the block group and census tract.

about GEOIDs: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
"""

BLOCKGROUP_GEOID = 'BLOCKGROUP GEOID'

print('NOTE: using limited area of boston, brookline, cambridge, somerville, so most datapoints will not fall in this area')
census_blockgroups_shapefile_filepath = "./shapefiles/ma/boston-brookline-cambridge-somerville_blockgroup.shp"
# census_tracts_shapefile_filepath = "./shapefiles/ma/msa_11460_census2010_blockgroups.shp"
blockgroups = geopandas.read_file(census_tracts_shapefile_filepath)
blockgroups = blockgroups.to_crs(epsg=4326)
blockgroups.plot()


def get_census_blockgroup(row):
    if random.randint(1, 1000) == 1:
        print(datetime.now(), "get census blockgroup")
    lat = float(row['lat'])
    lon = float(row['lon'])
    point = Point(lon, lat) # the points actually reverse - yes this looks weird but consider lon the x-axis and lat the y-axis on an x-y plane...
    for i, blockgroups_row in blockgroups.iterrows():
        try:
            # this could be a Polygon or a Multipolygon when the tract contains islands (<3 New England)
            blockgroups_polygon = blockgroups_row['geometry']
            if point.within(blockgroups_polygon):
                return blockgroups_row['GEOID10']
        except Exception as e:
            print('Exception when looking for point in census block group ' + blockgroups_row['GEOID10'], ': ', e)
            raise e
    return None


filtered_nighttime_df[BLOCKGROUP_GEOID] = nighttime_df.apply(get_census_blockgroup, axis=1)

In [ ]:
# Attach the tract GEOID from the blockgroup GEOID
TRACT_GEOID = 'TRACT GEOID'

def blockgroup_geoid_to_tract_geoid(blockgroup_geoid):
    blockgroup_geoid = str(blockgroup_geoid)
    return blockgroup_geoid[:-1]

filtered_nighttime_df[TRACT_GEOID] = filtered_nighttime_df[BLOCKGROUP_GEOID].apply(blockgroup_geoid_to_tract_geoid)

In [ ]:
# Drop the rows for datapoints where census area could not be determined
rows_before = filtered_nighttime_df.shape[0]
users_count_before = filtered_nighttime_df[DEVICE_ID].nunique()
filtered_nighttime_df.dropna(inplace=True)
rows_after = filtered_nighttime_df.shape[0]
users_count_after = filtered_nighttime_df[DEVICE_ID].nunique()
print("dropped %s rows where census area not determined. rows remaining: %s" % (rows_before - rows_after, rows_after))
print("users before: %s \nusers after: %s" % (users_count_before, users_count_after))

# Save data

In [ ]:
filtered_nighttime_filepath = data_path + "filtered/" + "filtered_" + "{msa}_{time_period}_nighttime_blockgroup_bos_brook_cam_som.csv".format(msa=MSA, time_period=TIME_PERIOD)

In [ ]:
print("saving data to file", filtered_nighttime_filepath)
filtered_nighttime_df.to_csv(filtered_nighttime_filepath)
# This is a stopping point - you can come back and read the data in from here.

In [ ]:
# open up the data if needed
# Read all previously filter data
print("reading data from file", filtered_nighttime_filepath)
filtered_nighttime_df = pd.read_csv(filtered_nighttime_filepath)
print(filtered_nighttime_df.shape)

In [ ]:
"""infer homes: (V1)
infer most likely home census area for user
choose geoid where the user dwells for the most time in the nighttime hours


- make map for each user as {geoid: cumulative nighttime dwelltime, for each geoid they are present at in nighttime hours}
- (where geoids are ids for census areas)
- then choose the geoid with the greatest cumulative nighttime dwelltime
"""

inferred_homes_map = {}
user_ids = filtered_nighttime_df[DEVICE_ID].unique()
print('inferring home areas for %s users' % len(user_ids))
for i, user_id in np.ndenumerate(user_ids):
    if (i[0] % 1000) == 0:
        print(i[0], datetime.now())
    # Make map for user as {geoid: cumulative dwelltime, for each geoid they are present at in nighttime hours}
    user_map = {}
    # Get the mapping of geoids to dwelltimes for given user
    u_df = filtered_nighttime_df[filtered_nighttime_df[DEVICE_ID] == user_id][[BLOCKGROUP_GEOID, NIGHTTIME_DWELLTIME]]
    for r, row in u_df.iterrows():
        geoid = row[BLOCKGROUP_GEOID]
        nighttime_dwelltime = row[NIGHTTIME_DWELLTIME]
        if geoid not in user_map:
            user_map[geoid] = 0
        user_map[geoid] += nighttime_dwelltime
    
    # choose the geoid with the greatest dwelltime
    max_dwelltime = 0
    max_dwelltime_geoid = None
    for geoid, nighttime_dwelltime in user_map.items():
        if nighttime_dwelltime > max_dwelltime:
            max_dwelltime = nighttime_dwelltime
            max_dwelltime_geoid = geoid
        
    inferred_homes_map[user_id] = max_dwelltime_geoid

print("made inferred homes map for %s users" % len(inferred_homes_map))

In [ ]:
"""Add inferred home area as colum in the larger (nighttime) DF."""

INFERRED_HOME_TRACT_GEOID = "inferred home census tract geoid"
INFERRED_HOME_BLOCKGROUP_GEOID = "inferred home census blockgroup geoid"

filtered_nighttime_df[INFERRED_HOME_BLOCKGROUP_GEOID] = filtered_nighttime_df[DEVICE_ID].map(inferred_homes_map)

In [ ]:
# Save this dataset
filtered_nighttime_filepath = data_path + "filtered/" + "filtered_" + "{msa}_{time_period}_inferred_homes_blockgroup_bos_brook_cam_som.csv".format(msa=MSA, time_period=TIME_PERIOD)
print("saving data to file", filtered_nighttime_filepath)
filtered_nighttime_df.to_csv(filtered_nighttime_filepath)
# This is a stopping point - you can come back and read the data in from here.

In [ ]:
# Aggregate the users to home census area.
# i.e. map inferred home census area geoid to number of users
# make DF with columns: INFERRED_HOME_CENSUS_TRACT_GEOID, USER_COUNT

USER_COUNT = "deviceID count"

inferred_home_geoids = nighttime_df[INFERRED_HOME_TRACT_GEOID].unique()
inferred_home_geoids_user_count = []
for geoid in inferred_home_geoids:
    geoid_df = nighttime_df[nighttime_df[INFERRED_HOME_TRACT_GEOID] == geoid]
    user_count = geoid_df[DEVICE_ID].nunique()
    inferred_home_geoids_user_count.append(user_count)

inferred_homes_df = pd.DataFrame.from_dict({
    INFERRED_HOME_TRACT_GEOID: inferred_home_geoids,
    USER_COUNT: inferred_home_geoids_user_count,
})
print(inferred_homes_df.shape)